# Visualizing Sentence Embeddings

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector

# Make sure Neural Monkey is in your PYTHONPATH before importing it:
sys.path.append(os.path.expanduser("~/neuralmonkey"))

from neuralmonkey.experiment import Experiment
from neuralmonkey.run import load_runtime_config
from neuralmonkey import dataset

## 1. Load Neural Monkey model

In [ ]:
exp = Experiment("mmmt-en-de/experiment.ini")
exp.build_model()

## 2. Load data

In [ ]:
run_cfg = load_runtime_config("val-data.ini")
data = run_cfg.test_datasets[0]

## 3. Run model on data

In [ ]:
_, outputs = exp.run_model(data, batch_size=256)

In [ ]:
outputs['encoded'].shape

## 4. Visualize data in Projector

In [ ]:
# Save embeddings in TSV format
np.savetxt("mmmt-en-de/embeddings.tsv", outputs['encoded'], delimiter='\t')

In [ ]:
# Save source and output sentences in TSV format
with open("mmmt-en-de/metadata.tsv", "w") as f:
    # TSV header
    print("source\toutput", file=f)

    # Data
    for src_words, tgt_words in zip(data.get_series('source'), outputs['target']):
        src = ' '.join(src_words)
        tgt = ' '.join(tgt_words)
        print("{}\t{}".format(src, tgt), file=f)

In [ ]:
# Create Projector config object
config = projector.ProjectorConfig()
emb = config.embeddings.add()
emb.tensor_name = "sent_embeddings"
emb.tensor_path = "embeddings.tsv"
emb.metadata_path = "metadata.tsv"

# Write Projector config to TensorBoard
writer = tf.summary.FileWriter("mmmt-en-de")
projector.visualize_embeddings(writer, config)